<h1> Imports </h1>
<h1 style="color:green;">Importações das bibliotecas</h1>

In [8]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

<h1> Load Data</h1>
<h1 style="color:green;">Leitura da base de dados</h1>

In [9]:
df = pd.read_csv("./data/ceaps_2008_to_2022.csv")
df.shape

(302070, 12)

<h1> Data Cleaning </h1>
<h1 style="color:green;"> Limpeza dos Dados</h1>

<p>It is important to analyze the quality of the dataset for consistency of information and results at the end of the analysis.</p>
<p style="color:green;"> É importante analisar a qualidade dos dados, para que no final das análises as informações e resultados sejam consistentes.</p>

<details>
    <summary><h3>What can be done? <p style="color:green;">O que pode ser feito?</h3> </p> </summary>


* check for duplicates or irrelevant columns <p style="color:green;"> verifique se há duplicatas ou colunas irrelevantes </p>

* check missing values <p style="color:green;"> verifique os valores ausentes </p>

* check unique values <p style="color:green;"> verifique os valores únicos das colunas </p>

* check column type <p style="color:green;"> verifique os tipos dos campos </p>

* check min and max column values <p style="color:green;"> verifique os valores máximos e mínimos das colunas buscando alguma irreguraridade com a realidade </p>

* check entries like: Gender column: your entries are "F" or Femine. Note that both have the same meaning but different values, this needs to be addressed <p style="color:green;">  verifique os valores de entrada, por exemplo: existe a coluna genero: esse compo possui valores do tipo "F" e "Feminino". Ambos tem o mesmo significado, mas estão com valores distintos </p>
    
* check which variables make sense to analyze, which ones can be discarded <p style="color:green;"> verifique quais variáveis fazem sentido analisar e quais podem ser descartados </p>

* check if you need to rearrange any field <p style="color:green;"> verifique se há a necessidade de reorganizar algum campo, por exemplo desmbrar data e hora em um campo só de data, outro só de hora </p>
    
* make sure the information is reliable <p style="color:green;"> verifique se as informações são confiáveis </p>

</details>

In [10]:
df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,csv
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/22,03/01/2022,Despesa com pagamento de aluguel de imóvel par...,6000,2173614.0,2022
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,26.320.603/0001-64,INFORMANAHORA,000000000000310/A,04/01/2022,Despesa com divulgação da atividade parlamenta...,1500,2173615.0,2022
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,13.659.201/0001-47,LINHA PURPURA FOTO E VIDEO LTDA,107,14/01/2022,Despesa com produção de texto e edição de víde...,6000,2173616.0,2022
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,23.652.846/0001-01,ROBERTO GUTIERREZ DA ROCHA M.E.I.,187,18/01/2022,Divulgação da atividade parlamentar,1000,2173618.0,2022
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,08.941.827/0001-01,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,000000000001772/A,17/01/2022,Divulgação da atividade parlamentar,2000,2173617.0,2022


In [11]:
for column in df.columns:

    print(f"{column: <18} | N uniques{'_'*10}{df[column].nunique(): >7} | % Null{'_'*10}{ round(df[column].isnull().sum() * 100 / len(df), 4)}%")

ANO                | N uniques__________     12 | % Null__________0.0%
MES                | N uniques__________     12 | % Null__________0.0%
SENADOR            | N uniques__________    278 | % Null__________0.0%
TIPO_DESPESA       | N uniques__________      7 | % Null__________0.0%
CNPJ_CPF           | N uniques__________  24153 | % Null__________0.0%
FORNECEDOR         | N uniques__________  45921 | % Null__________0.0%
DOCUMENTO          | N uniques__________ 195863 | % Null__________6.8156%
DATA               | N uniques__________   4476 | % Null__________0.0%
DETALHAMENTO       | N uniques__________  94810 | % Null__________40.2751%
VALOR_REEMBOLSADO  | N uniques__________  85142 | % Null__________0.0%
COD_DOCUMENTO      | N uniques__________ 272692 | % Null__________0.0007%
csv                | N uniques__________     12 | % Null__________0.0%


In [12]:
df = df.drop(columns = ['DOCUMENTO', 'COD_DOCUMENTO'])

In [13]:
# Como só o cnpj tem o /, vamos colocar todos eles como jurídico
df.loc[df['CNPJ_CPF'].str.contains('/'), 'PESSOA_JURIDICA'] = 1

# Agora quem tem o pessoa jurídica == 1, terá nas outras posições NAN, 
# Logo dá para fazer o filtro e descobrir os cpfs pela presenção do caractere - e o valor NAN em pessoa_juridica
df.loc[ df['CNPJ_CPF'].str.contains('-') & df['PESSOA_JURIDICA'].isna() , 'PESSOA_FISICA'] = 1

df = df.drop(columns='CNPJ_CPF')

In [14]:
df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,FORNECEDOR,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,csv,PESSOA_JURIDICA,PESSOA_FISICA
0,2022,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",GILBERTO PISELO DO NASCIMENTO,03/01/2022,Despesa com pagamento de aluguel de imóvel par...,6000,2022,NaN,1.0
1,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,INFORMANAHORA,04/01/2022,Despesa com divulgação da atividade parlamenta...,1500,2022,1.0,NaN
2,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,LINHA PURPURA FOTO E VIDEO LTDA,14/01/2022,Despesa com produção de texto e edição de víde...,6000,2022,1.0,NaN
3,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,ROBERTO GUTIERREZ DA ROCHA M.E.I.,18/01/2022,Divulgação da atividade parlamentar,1000,2022,1.0,NaN
4,2022,1,ACIR GURGACZ,Divulgação da atividade parlamentar,RONDONIA DINÂMICA COM. E SERV. DE INFORMÁTICA ...,17/01/2022,Divulgação da atividade parlamentar,2000,2022,1.0,NaN
